In [1]:
import sys
import os
sys.path.append('..')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import glob

import utilities.functions as functions
from utilities.functions import (
    segmentacao_3_otimizada,
    retidos11,
    calcula_viabilidade_wide,
)

In [2]:
!pwd

/Users/maceli/ifood_case/code/notebook


In [3]:
!ls -la dados/gold/df_decil.parquet

-rw-r--r--@ 1 maceli  staff  7338 Nov 29 18:24 dados/gold/df_decil.parquet


In [4]:
df = pd.read_parquet("dados/gold/df_decil.parquet")
df.head()


,customer_id,is_target,decil_1,decil_12,num_pedidos_mes_1,num_pedidos_mes_12,total_amount_mes_1,total_amount_mes_12
0,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,decil 9,decil 6,8,2,273.00,100.00
1,000021924bf8192f64a7cc17ce7c0bc511cba907308f80...,target,decil 8,decil 8,3,2,173.90,119.90
2,000022ced2f3aa411b66ac41167b272ae528c4acdffeeb...,target,decil 3,decil 3,2,2,68.65,68.65
3,000032b594523c3f8868edee4f1577b157e115cd01ab31...,control,decil 5,decil 5,1,1,88.80,88.80
4,0000bb10fb47a1d6b2d73754ef383950ef536c77d07212...,control,decil 9,decil 9,2,2,188.19,188.19


In [5]:
df['decil_1'].unique()

array(['decil 9', 'decil 8', 'decil 3', 'decil 5', 'decil 7', 'decil 4',
       'decil 1'], dtype=object)

In [6]:
df['decil_1_num'] = df['decil_1'].str.extract(r'(\d+)').astype(int)
df['decil_12_num'] = df['decil_12'].str.extract(r'(\d+)').astype(int)
df['delta_decil'] = df['decil_12_num'] - df['decil_1_num']

In [7]:
df=segmentacao_3_otimizada(df)

In [8]:
tabela_segmento = (
    df
    .groupby(['segmento_mobilidade', 'direcao', 'is_target'])
    .agg(
        clientes=('customer_id', 'nunique'),
        decil_inicial_medio=('decil_1_num', 'mean'),
        decil_final_medio=('decil_12_num', 'mean'),
        pedidos_medios_mes_1=('num_pedidos_mes_1', 'mean'),
        pedidos_medios_mes_12=('num_pedidos_mes_12', 'mean'),
        ticket_medio_mes_1=('total_amount_mes_1', 'mean'),
        ticket_medio_mes_12=('total_amount_mes_12', 'mean')
    )
    .reset_index()
)




In [9]:
tabela_segmento.to_csv('../Resultados/tabela_segmento.csv', index=False)

Marcar segmentacao - baseada no amount

In [10]:
df['direcao'].unique()

array(['Upgrade', 'Estável', 'Downgrade'], dtype=object)

In [11]:
conditions = [
    (df['segmento_mobilidade'] == 'Alta Mobilidade') & (df['direcao'] == 'Upgrade'),
    (df['segmento_mobilidade'] == 'Estável') & (df['direcao'] == 'Estável'),
    (df['segmento_mobilidade'] == 'Mobilidade Moderada') & (df['direcao'] == 'Downgrade')
]

choices = [1, 2, 3]

df['categoria_segmento'] = np.select(conditions, choices, default=None)


resumo_categorias = (
    df
    .groupby(['categoria_segmento', 'is_target'])['customer_id']
    .nunique()
    .reset_index(name='num_clientes')
    .sort_values(['categoria_segmento', 'is_target'])
)

In [12]:
df['categoria_segmento'].unique()

array([1, 2, 3], dtype=object)

In [13]:
df11=df[df['categoria_segmento']==1]
df12=df[df['categoria_segmento']==2]
df13=df[df['categoria_segmento']==3]

In [14]:
df_1,clientes_retidos_1=retidos11(df11, mes0=12, mes1=1,pedidos=1)
print(clientes_retidos_1)
df_2,clientes_retidos_2=retidos11(df12, mes0=12, mes1=1,pedidos=1)
print(clientes_retidos_1)
df_3,clientes_retidos_3=retidos11(df13, mes0=12, mes1=1,pedidos=1)
print(clientes_retidos_3)

           retidos  base  taxa_retencao
is_target                              
control          1     1            1.0
target           2     2            1.0
           retidos  base  taxa_retencao
is_target                              
control          1     1            1.0
target           2     2            1.0
           retidos  base  taxa_retencao
is_target                              
control          1     1            1.0


In [15]:
resultados, agg = calcula_viabilidade_wide(df_1,
                                           mes_campanha=12,
                                           mes_seguinte=1,
                                           coupon_value=10.0,
                                           margin_rate=0.12)

print(resultados)
print(agg)

{'pedidos_por_cliente_camp_control': 2.0, 'pedidos_por_cliente_camp_target': 3.0, 'pedidos_por_cliente_seg_control': 8.0, 'pedidos_por_cliente_seg_target': 7.0, 'inc_pedidos_camp': 2.0, 'inc_pedidos_seg': -2.0, 'inc_pedidos_total': 0.0, 'inc_gmv_camp': -90.19999999999999, 'inc_gmv_seg': -223.89999999999998, 'inc_gmv_total': -314.09999999999997, 'margem_incremental': -37.69199999999999, 'base_target_camp': 2, 'custo_campanha': 20.0, 'lucro_incremental': -57.69199999999999, 'roi': -2.8846, 'coupon_value': 10.0, 'margin_rate': 0.12, 'mes_campanha': 12, 'mes_seguinte': 1}
           pedidos_campanha  gmv_campanha  pedidos_seguinte  gmv_seguinte  \
is_target                                                                   
control                   2         100.0                 8         273.0   
target                    6         109.8                14         322.1   

           clientes  
is_target            
control           1  
target            2  


In [16]:
resultados, agg = calcula_viabilidade_wide(df_2,
                                           mes_campanha=12,
                                           mes_seguinte=1,
                                           coupon_value=10.0,
                                           margin_rate=0.12)

print(resultados)
print(agg)

{'pedidos_por_cliente_camp_control': 1.6666666666666667, 'pedidos_por_cliente_camp_target': 1.75, 'pedidos_por_cliente_seg_control': 1.6666666666666667, 'pedidos_por_cliente_seg_target': 2.0, 'inc_pedidos_camp': 0.33333333333333304, 'inc_pedidos_seg': 1.333333333333333, 'inc_pedidos_total': 1.666666666666666, 'inc_gmv_camp': -228.37000000000006, 'inc_gmv_seg': -174.37000000000006, 'inc_gmv_total': -402.7400000000001, 'margem_incremental': -48.328800000000015, 'base_target_camp': 4, 'custo_campanha': 40.0, 'lucro_incremental': -88.32880000000002, 'roi': -2.2082200000000003, 'coupon_value': 10.0, 'margin_rate': 0.12, 'mes_campanha': 12, 'mes_seguinte': 1}
           pedidos_campanha  gmv_campanha  pedidos_seguinte  gmv_seguinte  \
is_target                                                                   
control                   5        390.99                 5        390.99   
target                    7        292.95                 8        346.95   

           clientes  
is_targ

In [17]:
df['categoria_segmento'].unique()

array([1, 2, 3], dtype=object)

In [18]:
resultados, agg = calcula_viabilidade_wide(df_3,
                                           mes_campanha=12,
                                           mes_seguinte=1,
                                           coupon_value=10.0,
                                           margin_rate=0.12)

print(resultados)
print(agg)

KeyError: 'target'